In [1]:
import pandas as pd
import os
from datetime import datetime
import calendar
from time import gmtime

In [8]:
# download qgis
# real-time 911 - location
# null values in Force in the location areas



In [3]:
#county = pd.read_csv("/Users/sarahburgart/galvanize/week7/capstone2/RDC_InventoryCoreMetrics_County_sfh.csv")

In [4]:
#historical = pd.read_csv("/Users/sarahburgart/galvanize/week7/capstone2/RDC_InventoryCoreMetrics_US_Hist.csv")

In [5]:
#saletolist = pd.read_csv("/Users/sarahburgart/galvanize/week7/capstone2/SaleToListRatio_State.csv")

In [6]:
#from os import listdir
#filelist = [f for f in os.listdir('data') if f.endswith('.csv')]
#filelist
#df = pd.read_csv(f"data/{filelist[0]}")
#lst = [f"{col}{filelist[0][:6]}" for col in df.columns]
#df.columns = lst
#df.head()
#df3 = pd.read_csv(f"data/{filelist[1]}")

In [9]:
'''for file in filelist[1:]:
    #print(f"{file}")
    pd.concat([df, pd.read_csv(f"data/{file}")], axis=1)
    lst = [f"{col}{filelist[0][:6]}" for col in df.columns]
    df.columns = lst'''

'for file in filelist[1:]:\n    #print(f"{file}")\n    pd.concat([df, pd.read_csv(f"data/{file}")], axis=1)\n    lst = [f"{col}{filelist[0][:6]}" for col in df.columns]\n    df.columns = lst'

In [2]:
# function to convert datetime object to 4 - for force ONLY
def datetime(df, col_as_str):
    yearmonthday = df[col_as_str].astype(str).str.split(" ")
    ymd = yearmonthday.apply(lambda x: x[0]).str.split("/")
    year = ymd.apply(lambda x: x[2])
    month = ymd.apply(lambda x: x[0])
    day = ymd.apply(lambda x: x[1])
    time = ymd = yearmonthday.apply(lambda x: x[1]+x[2]).astype(str)
    df["year"] = year
    df["month"] = month
    df["day"] = day
    df["time"] = time
    return df


In [3]:
def callstime(df, col_arrive):
    yearmonthday = df[col_arrive].astype(str).str.split(" ")
    df["year"] = yearmonthday.apply(lambda x: x[2])
    df["day"] = yearmonthday.apply(lambda x: x[1])
    month_dict = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec":12}
    df["month"] = yearmonthday.apply(lambda x: month_dict[x[0]])
    df["time"] = yearmonthday.apply(lambda x: x[3])
    return df
    
    

In [4]:
calls = pd.read_csv("/Users/sarahburgart/galvanize/week7/capstone2/data/Call_Data.csv")

In [5]:
force = pd.read_csv("/Users/sarahburgart/galvanize/week7/capstone2/data/Use_Of_Force.csv")

In [6]:
beats = pd.read_csv("/Users/sarahburgart/galvanize/week7/capstone2/data/Seattle_Police_Department_Beats.csv")

In [17]:
force = datetime(force,"Occured_date_time" )


In [9]:
foo = pd.to_datetime( force.Occured_date_time )
epoch_second = foo.map(lambda x:x.value/1e9)

In [10]:
force["epoch"] = epoch_second

In [12]:
calls = callstime(calls,"Arrived Time")


In [13]:
calls17 = calls[calls["year"].isin(["2014", "2015", "2016", "2017", "2018", "2019"])]

In [14]:
bar = pd.to_datetime( calls17["Arrived Time"], format="%b %d %Y %I:%M:%S:%f%p" )
epoch_second2 = bar.map(lambda x: x.value/1e9)

In [15]:
calls17["epoch_calls"] = epoch_second2

/Users/sarahburgart/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
# remove repetitive columns

calls_tosql = calls17.drop(columns =["year","day", "month", "time", "Precinct", "Sector", "Arrived Time"])
calls_tosql.set_index("epoch_calls", inplace=True)


In [29]:
# remove repetitive columns (incident type, officer id and subject id are in ID respectively)

force_sql = force.drop(columns=["Occured_date_time", "year", "month", "day","time", "Incident_Num", "Officer_ID",
                               "Subject_ID"])
